In [ ]:
import numpy as np
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# Khai báo trình duyệt Chrome
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Mở URL
driver.get("https://vnexpress.net/goc-nhin")
driver.maximize_window()

# Hàm cuộn trang
def scroll_page():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(random.uniform(2, 4))

# Lấy dữ liệu bài viết
def extract_articles():
    articles = driver.find_elements(By.CSS_SELECTOR, "article.item-news")
    data = []
    for article in articles:
        try:
            # Tiêu đề và link
            title_element = article.find_element(By.CSS_SELECTOR, "h3.title-news a")
            title = title_element.text
            link = title_element.get_attribute("href")

            # Mô tả
            description_element = article.find_element(By.CSS_SELECTOR, "p.description a")
            description = description_element.text

            # Tác giả
            author_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.name-author")
            author = author_element.text
            author_link = author_element.get_attribute("href")

            # Danh mục
            category_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.cat")
            category = category_element.text
            category_link = category_element.get_attribute("href")

            # Số lượng comment
            try:
                comment_element = article.find_element(By.CSS_SELECTOR, "p.meta-news .count_cmt span.txt_num_comment")
                comments = comment_element.text.strip()  # Lấy số comment nếu có
            except NoSuchElementException:
                comments = "0"  # Không có comment

            # Lưu dữ liệu
            data.append({
                "Title": title,
                "Link": link,
                "Description": description,
                "Author": author,
                "Author Link": author_link,
                "Category": category,
                "Category Link": category_link,
                "Comments": comments
            })
        except NoSuchElementException:
            # Nếu không tìm thấy phần tử, bỏ qua
            continue
    return data

# Cuộn và lấy dữ liệu
all_data = []
for _ in range(5):  # Cuộn 5 lần
    scroll_page()
    articles_data = extract_articles()
    all_data.extend(articles_data)

# Loại bỏ trùng lặp
df = pd.DataFrame(all_data).drop_duplicates()

# Lưu vào file Excel
df.to_excel("D:/du lieu o cu/HUTECH Courses/Social Networking Course/SocialNetworkingProject/Project của Đạt/vnexpress_articles.xlsx")
print("Dữ liệu đã được lưu vào file vnexpress_articles_with_comments.xlsx")

# Đóng trình duyệt
driver.quit()


TypeError: NDFrame.to_excel() got an unexpected keyword argument 'encoding'